In [3]:
# ##google colab用にchainerのインストール
# !curl https://colab.chainer.org/install | sh -

In [4]:
# ##google driveのマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
import numpy as np
import chainer

from chainer import cuda,Function,report,training,utils,Variable
from chainer import datasets,iterators,optimizers,serializers
from chainer import Link,Chain,ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import tuple_dataset
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp
from hyperopt.pyll import scope


%matplotlib inline

In [6]:
##GPU環境の設定を確認
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [7]:
##colab用にデータのpathを指定
##My Driveのカンマはpath指定の時にはいらない

#DATA_PATH = "/content/drive/My Drive/resarch"
#!ls /content/drive/'My Drive'/resarch/

DATA_PATH='./'

In [8]:
ls *.txt

train_data201604_without_normalize_2.txt  train_user_id201606.txt
train_data201606_without_normalize.txt    train_user_id201606_2.txt
train_data201606_without_normalize_2.txt  x_test_corr_high.txt
train_label201604_2.txt                   x_train_corr_high.txt
train_label201606.txt                     y_test_corr_high.txt
train_label201606_2.txt                   y_train_corr_high.txt
train_user_id201604_2.txt


In [9]:
##データの読み込み
import pickle

## train data
f=open(DATA_PATH+"/x_train_corr_high.txt","rb")
pre_data_train=pickle.load(f)
f=open(DATA_PATH+"/y_train_corr_high.txt","rb")
label_train=pickle.load(f)

### 不均衡データなので、
#### 1,正解データと不例データから均等にサンプリングして複数データセットをひとつにまとめたものをデータセットにする<br>
#### 2,アンダーサンプリングで、正例データの数に不例データの数を合わせる

In [10]:
##validationデータを正例、不例の一定割合データセットに分ける
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in range(len(label_train)):
    if label_train[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
##train時にはバッチサイズ64として、正例と不例を32ずつindexをサンプリングしたデータセットを複数作成して、まとめてデータセットとする
##正例、不例一定割合のデータセット作成
x=[]
y=[]

'''make train data'''
'''1,正解データと不例データから均等にサンプリングして複数データセットをひとつにまとめたものをデータセットにする'''
for i in range(2000):
    posi=np.random.choice(buy,32,replace = False)
    nega=np.random.choice(no_buy,32,replace = False)
    for i in posi:
        x.append(pre_data_train[i])
        y.append(label_train[i])
    for i in nega:
        x.append(pre_data_train[i])
        y.append(label_train[i])

    
# '''2,アンダーサンプリングで、正例データの数に不例データの数を合わせる'''
# nega=np.random.choice(no_buy,len(buy),replace = False)
# for i in buy:
#     x.append(pre_data_train[i])
#     y.append(label_train[i])
# for i in nega:
#     x.append(pre_data_train[i])
#     y.append(label_train[i])
    

**hyperoptで調べたいこと<br>
・convolutionの数、チャネルの数、filaの数、padの数<br>
・最適化関数の設定**<br>
※フィルタサイズとパディングは固定して、いくつか試して良い物を選択する。仮に複数フィルタサイズを試したいなら、for文を積むように書いて行う


In [42]:
#ネットワーク定義

#out_chanel=50
class Purchase(Chain):
    def __init__(self,out_chanel,mid_units,conv_num,activate):
        super(Purchase , self).__init__(
            ##k_size:5
            conv1=L.Convolution2D(1,out_chanel,ksize=5,pad=2),
            conv2=L.Convolution2D(out_chanel,out_chanel,ksize=5,pad=2),
            ##k_size:3
            conv3=L.Convolution2D(out_chanel,out_chanel,ksize=3,pad=1),
            conv4=L.Convolution2D(out_chanel,out_chanel,ksize=3,pad=1),
            conv5=L.Convolution2D(out_chanel,out_chanel,ksize=3,pad=1),
            bn1=L.BatchNormalization(out_chanel),
            bn2=L.BatchNormalization(out_chanel),
            bn3=L.BatchNormalization(out_chanel),
            bn4=L.BatchNormalization(out_chanel),
            bn5=L.BatchNormalization(out_chanel),
            l1=L.Linear(None,mid_units),
            l2=L.Linear(mid_units,2)
           
        
        )
        self.conv_num = conv_num
        if activate == 'relu':
            self.act = F.relu
        elif activate == 'tanh':
            self.act = F.tanh
        elif activate == 'sigmoid':
            self.act = F.sigmoid
        
    def __call__(self,x):
            
            h1=self.act(self.bn1(self.conv1(x)))
            h2=self.act(self.bn2(self.conv2(h1)))
            if self.conv_num == 2:
                h=F.dropout(self.act(self.l1(h2)))
                return self.l2(h)
            h3=self.act(self.bn3(self.conv3(h2)))
            if self.conv_num == 3:
                h=F.dropout(self.act(self.l1(h3)))
                return self.l2(h)
            h4=self.act(self.bn4(self.conv4(h3)))
            if self.conv_num == 4:
                h=F.dropout(self.act(self.l1(h4)))
                return self.l2(h)
            h5=self.act(self.bn5(self.conv5(h4)))
            if self.conv_num == 5:
                h=F.dropout(self.act(self.l1(h5)))
                return self.l2(h)

In [43]:
def main(params):
    epoch = 20
    gpu = 0
    batchsize = 64

    out_chanel = params['out_chanel']
    mid_units = params['mid_units']
    conv_num = params['conv_num']
    activate = params['activate']
    optimizer_name = params['optimizer_name']
    lr = params['lr']
    weight=params['weight']

    model = L.Classifier(Purchase(out_chanel, mid_units, conv_num, activate))
    
    if gpu >= 0:
        chainer.cuda.get_device(gpu).use()
        model.to_gpu()

    # Setup an optimizer
    if optimizer_name == 'Adam':
        optimizer = optimizers.Adam()
    elif optimizer_name == 'AdaDelta':
        optimizer = optimizers.AdaDelta()
    else:
        optimizer = optimizers.MomentumSGD(lr=lr)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(weight))
    
    ##load data
    ##x,yのデータをtuple化して、更にvalidation dataを作成する

    data= tuple_dataset.TupleDataset(x,y)
    split_at=int(len(x)*0.95)
    train,test=chainer.datasets.split_dataset(data ,split_at)
    
    ##trainer
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    test_iter = chainer.iterators.SerialIterator(test, batchsize,repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.StandardUpdater(train_iter, optimizer, device=gpu)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="hyperopt_result")
    
    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Save two plot images to the result dir
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                               'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
             'epoch', file_name='accuracy.png'))

    # Write report
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())

    # Run the training
    trainer.run()
    valid_data = trainer._extensions['PlotReport'].extension._data
    loss_data = [data for i, data in valid_data['validation/main/loss']]
    best10_loss = sorted(loss_data)[:10]
    return sum(best10_loss)

In [44]:
if __name__ == '__main__':
    ##探索空間の設定
    space = {
        'out_chanel':scope.int(hp.quniform('out_chanel',100,300,50)),
        'mid_units':scope.int(hp.quniform('mid_units',50,300,50)),
        'conv_num':scope.int(hp.quniform('conv_num',2,5,1)),
        'activate':hp.choice('activate',('relu','tanh','sigmoid')),
        'optimizer_name':hp.choice('optimizer_name',('Adam','AdaDelta','MomentumSGD')),
        'lr':hp.uniform('lr',0.005,0.02),
        'weight':hp.uniform('weight',0.0001,0.005),
        
    }
    best = fmin(main, space, algo=tpe.suggest, max_evals=100)
    print("best parameters",best)

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  0.26%
this epoch [##................................................]  5.26%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.53%
this epoch [#####.............................................] 10.53%
       200 iter, 0 epoch / 20 epochs
    16.661 iters/sec. Estimated time to finish: 0:37:48.798843.
     total [..................................................]  0.79%
this epoch [#######...........................................] 15.79%
       300 iter, 0 epoch / 20 epochs
    17.376 iters/sec. Estimated time to finish: 0:36:09.718410.
     total [..................................................]  1.05%
this epoch [##########........................................] 21.05%
       400 iter, 0 epoch / 20 epochs

     total [####..............................................]  8.68%
this epoch [####################################..............] 73.68%
      3300 iter, 1 epoch / 20 epochs
    17.476 iters/sec. Estimated time to finish: 0:33:05.566054.
     total [####..............................................]  8.95%
this epoch [#######################################...........] 78.95%
      3400 iter, 1 epoch / 20 epochs
     17.48 iters/sec. Estimated time to finish: 0:32:59.434955.
     total [####..............................................]  9.21%
this epoch [##########################################........] 84.21%
      3500 iter, 1 epoch / 20 epochs
    17.483 iters/sec. Estimated time to finish: 0:32:53.328947.
     total [####..............................................]  9.47%
this epoch [############################################......] 89.47%
      3600 iter, 1 epoch / 20 epochs
    17.484 iters/sec. Estimated time to finish: 0:32:47.507722.
     total [####............

     total [########..........................................] 17.11%
this epoch [#####################.............................] 42.11%
      6500 iter, 3 epoch / 20 epochs
    17.287 iters/sec. Estimated time to finish: 0:30:22.171659.
     total [########..........................................] 17.37%
this epoch [#######################...........................] 47.37%
      6600 iter, 3 epoch / 20 epochs
    17.291 iters/sec. Estimated time to finish: 0:30:15.940499.
     total [########..........................................] 17.63%
this epoch [##########################........................] 52.63%
      6700 iter, 3 epoch / 20 epochs
    17.301 iters/sec. Estimated time to finish: 0:30:09.151607.
     total [########..........................................] 17.89%
this epoch [############################......................] 57.89%
      6800 iter, 3 epoch / 20 epochs
    17.311 iters/sec. Estimated time to finish: 0:30:02.306335.
     total [#########.......

     total [############......................................] 25.53%
this epoch [#####.............................................] 10.53%
      9700 iter, 5 epoch / 20 epochs
     17.38 iters/sec. Estimated time to finish: 0:27:08.328544.
     total [############......................................] 25.79%
this epoch [#######...........................................] 15.79%
      9800 iter, 5 epoch / 20 epochs
    17.386 iters/sec. Estimated time to finish: 0:27:02.015596.
     total [#############.....................................] 26.05%
this epoch [##########........................................] 21.05%
      9900 iter, 5 epoch / 20 epochs
    17.392 iters/sec. Estimated time to finish: 0:26:55.713191.
     total [#############.....................................] 26.32%
this epoch [#############.....................................] 26.32%
     10000 iter, 5 epoch / 20 epochs
    17.398 iters/sec. Estimated time to finish: 0:26:49.407330.
     total [#############...

     total [#################.................................] 34.21%
this epoch [##########################################........] 84.21%
     13000 iter, 6 epoch / 20 epochs
    17.492 iters/sec. Estimated time to finish: 0:23:49.238275.
     total [#################.................................] 34.47%
this epoch [############################################......] 89.47%
     13100 iter, 6 epoch / 20 epochs
    17.496 iters/sec. Estimated time to finish: 0:23:43.191140.
     total [#################.................................] 34.74%
this epoch [###############################################...] 94.74%
     13200 iter, 6 epoch / 20 epochs
      17.5 iters/sec. Estimated time to finish: 0:23:37.149473.
7           0.6684      0.669805              0.593257       0.586094                  762.975       
     total [#################.................................] 35.00%
this epoch [..................................................]  0.00%
     13300 iter, 7 epoch / 

...

     total [############################......................] 57.63%
this epoch [##########################........................] 52.63%
     21900 iter, 11 epoch / 20 epochs
    7.3593 iters/sec. Estimated time to finish: 0:36:27.711159.
     total [############################......................] 57.89%
this epoch [############################......................] 57.89%
     22000 iter, 11 epoch / 20 epochs
    7.3593 iters/sec. Estimated time to finish: 0:36:14.132634.
     total [#############################.....................] 58.16%
this epoch [###############################...................] 63.16%
     22100 iter, 11 epoch / 20 epochs
    7.3592 iters/sec. Estimated time to finish: 0:36:00.556292.
     total [#############################.....................] 58.42%
this epoch [##################################................] 68.42%
     22200 iter, 11 epoch / 20 epochs
    7.3591 iters/sec. Estimated time to finish: 0:35:46.999519.
     total [############

     total [#################################.................] 66.05%
this epoch [##########........................................] 21.05%
     25100 iter, 13 epoch / 20 epochs
    7.2909 iters/sec. Estimated time to finish: 0:29:29.325729.
     total [#################################.................] 66.32%
this epoch [#############.....................................] 26.32%
     25200 iter, 13 epoch / 20 epochs
    7.3094 iters/sec. Estimated time to finish: 0:29:11.162963.
     total [#################################.................] 66.58%
this epoch [###############...................................] 31.58%
     25300 iter, 13 epoch / 20 epochs
    7.2959 iters/sec. Estimated time to finish: 0:29:00.710404.
     total [#################################.................] 66.84%
this epoch [##################................................] 36.84%
     25400 iter, 13 epoch / 20 epochs
    7.2889 iters/sec. Estimated time to finish: 0:28:48.648408.
     total [############

     total [#####################################.............] 74.74%
this epoch [###############################################...] 94.74%
     28400 iter, 14 epoch / 20 epochs
    7.2745 iters/sec. Estimated time to finish: 0:21:59.681482.
15          0.697882    0.694439              0.499293       0.5                       3892.83       
     total [#####################################.............] 75.00%
this epoch [..................................................]  0.00%
     28500 iter, 15 epoch / 20 epochs
    7.2444 iters/sec. Estimated time to finish: 0:21:51.358507.
     total [#####################################.............] 75.26%
this epoch [##................................................]  5.26%
     28600 iter, 15 epoch / 20 epochs
    7.2494 iters/sec. Estimated time to finish: 0:21:36.658120.
     total [#####################################.............] 75.53%
this epoch [#####.............................................] 10.53%
     28700 iter, 15 epoc

     total [#########################################.........] 83.16%
this epoch [###############################...................] 63.16%
     31600 iter, 16 epoch / 20 epochs
    7.2728 iters/sec. Estimated time to finish: 0:14:39.990129.
     total [#########################################.........] 83.42%
this epoch [##################################................] 68.42%
     31700 iter, 16 epoch / 20 epochs
    7.2727 iters/sec. Estimated time to finish: 0:14:26.252997.
     total [#########################################.........] 83.68%
this epoch [####################################..............] 73.68%
     31800 iter, 16 epoch / 20 epochs
    7.2727 iters/sec. Estimated time to finish: 0:14:12.499855.
     total [#########################################.........] 83.95%
this epoch [#######################################...........] 78.95%
     31900 iter, 16 epoch / 20 epochs
    7.2727 iters/sec. Estimated time to finish: 0:13:58.754526.
     total [############

KeyboardInterrupt: 